In [9]:
from sklearn.metrics import classification_report
from tensorflow.keras.datasets.fashion_mnist import load_data as load_data_Fashion_MNIST
from tensorflow.keras.datasets.cifar10 import load_data as load_data_CIFAR_10

from nn.losses import *
from nn.layers import *
from nn.initializers import *
from nn.activations import *
from nn.network import NeuralNetwork, MulticlassNNClassifier

## Reloading imports

In [3]:
%load_ext autoreload

In [4]:
%autoreload

## Loading datasets

In [10]:
(x_train_fmnist, y_train_fmnist), (x_test_fmnist, y_test_fmnist) = load_data_Fashion_MNIST()
(x_train_cifar10, y_train_cifar10), (x_test_cifar10, y_test_cifar10) = load_data_CIFAR_10()

fmnist = {
    'train': {
        'x': x_train_fmnist,
        'y': y_train_fmnist
    },
    'test': {
        'x': x_test_fmnist,
        'y': y_test_fmnist
    }
}

cifar10 = {
    'train': {
        'x': x_train_cifar10,
        'y': y_train_cifar10
    },
    'test': {
        'x': x_test_cifar10,
        'y': y_test_cifar10
    }
}

# https://www.kaggle.com/datasets/boulahchichenadir/algerian-used-cars

170498071/170498071 [==============================] - 40s 0us/step


## Fashion MNIST

### Tensorflow

### This library

## CIFAR 10

### Tensorflow

### This library

## 20 class dataset

### Tensorflow

### This library